# Chapter1

### 主机名-->IP地址

In [2]:
import socket
hostname = 'maps.google.com'
addr = socket.gethostbyname(hostname)
print('The IP address of {} is {}'.format(hostname, addr))

The IP address of maps.google.com is 108.177.103.163


### IP地址
IPv4 4字节IP
  
    `127.*.*.*` 预留本地程序使用
  
    `127.0.0.1` localhost
     
    IP地址与掩码表示子网
    
    127.0.0.0/8 前8位为127，后面任意值
    
    192.168.0.0/16
    
    192.168.5.0/24
  
IPv6 16字节

# Chapter2 UDP

## 端口号
16bits 无符号数 0~65536

Source (IP : port number) --> Destination (IP : port number)

DNS 默认端口53  0~1023 分配给最重要服务

使用`more /etc/services`来查看

In [6]:
socket.getservbyname("domain") #查询域名服务的端口号

53

## 套接字
[python--threading多线程总结](https://www.cnblogs.com/tkqasn/p/5700281.html)

```python
#!/usr/bin/env python3
# Foundations of Python Network Programming, Third Edition
# https://github.com/brandon-rhodes/fopnp/blob/m/py3/chapter02/udp_local.py
# UDP client and server on localhost

import argparse, socket
from datetime import datetime

MAX_BYTES = 65535

def server(port):
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.bind(('127.0.0.1', port))
    print('Listening at {}'.format(sock.getsockname()))
    while True:
        data, address = sock.recvfrom(MAX_BYTES)
        text = data.decode('ascii')
        print('The client at {} says {!r}'.format(address, text))
        text = 'Your data was {} bytes long'.format(len(data))
        data = text.encode('ascii')
        sock.sendto(data, address)

def client(port):
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    text = 'The time is {}'.format(datetime.now())
    data = text.encode('ascii')
    sock.sendto(data, ('127.0.0.1', port))
    print('The OS assigned me the address {}'.format(sock.getsockname()))
    data, address = sock.recvfrom(MAX_BYTES)  # Danger! See Chapter 2
    text = data.decode('ascii')
    print('The server {} replied {!r}'.format(address, text))

if __name__ == '__main__':
    choices = {'client': client, 'server': server}
    parser = argparse.ArgumentParser(description='Send and receive UDP locally')
    parser.add_argument('role', choices=choices, help='which role to play')
    parser.add_argument('-p', metavar='PORT', type=int, default=1060,
                        help='UDP port (default 1060)')
    args = parser.parse_args()
    function = choices[args.role]
    function(args.p)
```

In [2]:
import threading
threads=[]
t1=threading.Thread(target=server,args=(1065,))
t2=threading.Thread(target=client,args=(1065,))
t3=threading.Thread(target=client,args=(1065,))

threads.append(t1)
threads.append(t2)
threads.append(t3)

for t in threads:
    t.setDaemon(True)
    t.start()

Listening at ('127.0.0.1', 1065)
The OS assigned me the address ('0.0.0.0', 36341)
The client at ('127.0.0.1', 47420) says 'The time is 2018-10-11 18:08:18.474081'
The OS assigned me the address ('0.0.0.0', 47420)
The server ('127.0.0.1', 1065) replied 'Your data was 38 bytes long'


In [3]:
threading.active_count()

7

In [4]:
socket.has_ipv6

True

# Chapter 4 套接字名与DNS

## 4.1 主机名与套接字

#### 主要socket方法